# Importing library

In [ ]:
!pip install keras

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

# Reading Dataframe

In [ ]:
review=pd.read_csv("E:\\DSC\\PROJECTS\\sentiment analysis\\Reviews.csv")
review.head()

In [ ]:
review.tail()

In [ ]:
print('The number of entries in the data frame: ', review.shape[0])

In [ ]:
review['ProductId'].nunique()

In [ ]:
review['UserId'].nunique()

In [ ]:
review.isnull().sum()

In [ ]:
review.dropna(inplace=True)

In [ ]:
review.isnull().sum()

# Data Analysis

In [ ]:
fig = px.histogram(review, x="Score")
fig.update_traces(marker_color="orange",marker_line_color='rgb(8,48,107)',
                  marker_line_width=2.0)
plt.figure(figsize=(5,3))
fig.update_layout(title_text='Product Score')
fig.show()

from this graph we can say that most of the reviews are postive.score is the ratings given by the customer

In [ ]:
review = review[review['Score'] != 3]
review['sentiment'] = review['Score'].apply(lambda rating : +1 if rating > 3 else -1)
positive = review[review['sentiment'] == 1]
negative = review[review['sentiment'] == -1]
review.head()

In [ ]:
review['sentimentt'] = review['sentiment'].replace({-1 : 'negative'})
review['sentimentt'] = review['sentimentt'].replace({1 : 'positive'})
fig = px.histogram(review, x="sentimentt")
fig.update_traces(marker_color="indianred",marker_line_color='rgb(10,48,107)',
                  marker_line_width=1.5)
fig.update_layout(title_text='Product Sentiment')
fig.show()

In [ ]:
review.head()

In [ ]:
X=data['Score']
y=data["sentiment"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.80, random_state=0)

from sklearn.preprocessing import StandardScaler
ss_train = StandardScaler()
X_train = ss_train.fit_transform(X_train)
ss_test = StandardScaler()
X_test = ss_test.fit_transform(X_test)

from sklearn.svm import LinearSVC
svm = LinearSVC(max_iter=10000).fit(X_train, y_train)

svm_predict = svm.predict(X_test)
from sklearn.metrics import classification_report
print(classification_report(y_test, svm_predict))

In [ ]:
df=pd.read_csv("E:\\DSC\\PROJECTS\\sentiment analysis\\Reviews.csv")
df.head()


In [ ]:
dt=df
label_encoders={}
categorical_coloums= dt.columns
for columns in categorical_coloums:
    label_encoders[columns]=LabelEncoder()
    dt[columns]=label_encoders[columns].fit_transform(dt[columns])

# Data Cleaning

In [ ]:
review = review.drop(['ProductId','UserId','ProfileName','Id','HelpfulnessNumerator','HelpfulnessDenominator','Score','Time','Summary'], axis=1)
review.info(memory_usage='deep')

In [ ]:
review.head()

# Split the DataFrame

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(review['Text'], review['sentiment'], random_state = 0)
print('X_train first entry: \n\n', X_train[0])
print('\n\nX_train shape: ', X_train.shape)

# Create a bag of words

Next, we will use a count vectorizer from the Scikit-learn library.
This will transform the text in our data frame into a bag of words model, which will contain a sparse matrix of integers. The number of occurrences of each word will be counted and printed.
We will need to convert the text into a bag-of-words model since the logistic regression algorithm cannot understand text.

In [ ]:
vect = CountVectorizer().fit(X_train)
vect

In [ ]:
# checking the features
feat = vect.get_feature_names()

In [ ]:
len(vect.get_feature_names())

# Sparse matrix

In [ ]:
X_train_vectorized = vect.transform(X_train)
# the interpretation of the columns can be retreived as follows
# X_train_vectorized.toarray()

In [ ]:
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

In [ ]:
# accuracy
predictions = model.predict(vect.transform(X_test))

In [ ]:
accuracy_score(y_test, predictions)

In [ ]:
# area under the curve
roc_auc = roc_auc_score(y_test, predictions)
print('AUC: ', roc_auc)
fpr, tpr, thresholds = roc_curve(y_test, predictions)

In [ ]:
plt.title('ROC for logistic regression on bag of words', fontsize=20)
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate', fontsize = 20)
plt.xlabel('False Positive Rate', fontsize = 20)
plt.show()

In [ ]:
# coefficient determines the weight of a word (positivity or negativity)
# checking the top 10 positive and negative words

# getting the feature names
feature_names = np.array(vect.get_feature_names())

# argsort: Integer indicies that would sort the index if used as an indexer
sorted_coef_index = model.coef_[0].argsort()

print('Smallest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:10]]))
print('Largest Coefs: \n{}\n'.format(feature_names[sorted_coef_index[:-11:-1]]))


# Multinomial Naive Bayes Algorithm

In [ ]:
model=MultinomialNB()
model.fit(X_train_vectorized,y_train)

In [ ]:
predictions=model.predict(vect.transform(X_test))
roc_auc = roc_auc_score(y_test, predictions)
print('AUC: ', roc_auc)

It gives us a score of 0.844 which is good.

# Bigrams

In [ ]:
vect = CountVectorizer(min_df = 5, ngram_range = (1,2)).fit(X_train)

In [ ]:
X_train_vectorized = vect.transform(X_train)

In [ ]:
len(vect.get_feature_names())

In [ ]:
feat = vect.get_feature_names()

In [ ]:
# the number of features has increased again
# checking for the AUC
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

In [ ]:
predictions = model.predict(vect.transform(X_test))

In [ ]:
accuracy_score(y_test, predictions)

In [ ]:
roc_auc = roc_auc_score(y_test, predictions)
print('AUC: ', roc_auc)
fpr, tpr, thresholds = roc_curve(y_test, predictions)

In [ ]:
plt.title('ROC for logistic regression on Bigrams', fontsize=20)
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate', fontsize = 20)
plt.xlabel('False Positive Rate', fontsize = 20)
plt.show()

In [ ]:
# check the top 10 features for positive and negative
# reviews again, the AUC has improved
feature_names = np.array(vect.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()

# print('Smallest Coef: \n{}\n'.format(feature_names[sorted_coef_index][:10]))
# print('Largest Coef: \n{}\n'.format(feature_names[sorted_coef_index][:-11:-1]))

In [ ]:
new_review = ['The food is not good, I would never buy them again']
print(model.predict(vect.transform(new_review)))

In [ ]:
new_review = ['One would not be disappointed by the food']
print(model.predict(vect.transform(new_review)))